In [3]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 41.1 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = nn.Dropout(p=0.5)(x)  # Apply dropout with probability 0.5
        x = self.fc2(x)
        return torch.softmax(x, dim=1)


# Prepare MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# Train the model for 1 epoch
train(1)

# Export the model to ONNX
dummy_input = torch.randn(1, 1, 28, 28)  # Create a dummy input
output_onnx = 'mnist_model.onnx'
torch.onnx.export(model, dummy_input, output_onnx)

print(f"Model has been exported to {output_onnx}")


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306765
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.304964
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.303008
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.306467
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300498
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.303581
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.300709
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.305306
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.299286
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.299572
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.301690
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.303046
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.303776
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.307018
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.299914
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.303836
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.300765
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.297573
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.295887
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.299741
Train Epoch: 1 [12800/60000 (